In [ ]:

from neuralogic.nn import get_evaluator
from neuralogic.core import R, Template, V, Settings, Transformation, Combination, Aggregation
from neuralogic.core.constructs.function.tree import F
from neuralogic.core.settings import Settings
from neuralogic.dataset import Dataset
from neuralogic.optim import SGD


train_dataset = Dataset()
template = Template()

train_dataset.add_example([R.b[3], R.c[5]])

# just write to template list of operations to perform
template += (R.a <= (R.b, R.c)) | [Combination.PRODUCT, Transformation.IDENTITY, Aggregation.AVG]
template += (R.a <= (x:=R.b, y:= R.c)) >> F.avg[F.identity(x * y)]
template += (R.a <= F.avg[F.identity(R.b * R.c)])

# template += (rule with metadata)
# a = b + c


#template += (R.a <= (x:=R.b, y:= R.c)) >> F.identity(x + y)
# expression above has following eval order:
# R.a    ||   x:= R.b    ||     y:= R.c    ||     R.a <= ()   ||    x + y   ||    F.identity  ||    >>   ||   template +=


# now useless
train_dataset.add_queries([R.a[3]])

# [R.relA(V.X) <= (R.relB(V.Y), R.relC(V.Y, V.X))] | [Transformation.SIGMOID]
# (R.message2(V.X)[5,10] <= (R.message1(V.Y)[10,20], R.edge(V.Y, V.X))) | [Transformation.RELU, Aggregation.AVG]
# (R.m2(V.X) <= [x:=R.m1(V.X), y:=R.edge(V.Y, V.X)]) >> F.sigmoid(x + y)

settings = Settings(optimizer=SGD(), epochs=1)
# calling evaluator with template with rules that have no metadata
neuralogic_evaluator = get_evaluator(template, settings)

build_dataset = neuralogic_evaluator.build_dataset(train_dataset)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

In [ ]:
build_dataset[0].draw()